# Prácticas

_Máster Big Data Analytics 2015/2016_  
_Luis Belloch luisbelloch@gmail.com_  
_Universidad Politécnica de Valencia_  

El objetivo de la práctica es aprender a trasladar a Spark pequeños prototipos de algoritmos de cálculo que previamente hemos visto en clase mediante Pandas.

Es importante señalar que la implementación de este documento sólo se incluye como referencia para poder entender los cálculos. Para los ejercicios únicamente puede utilizarse `PySpark` y la librería estándar de Python. Opcionalmente puedes hacer uso tambien de `Spark SQL`, pero no se permite instalar terceras librerías a no ser que el ejercicio lo indique.

Los ejercicios específicos que debe realizar cada grupo son:

* Diploma: 1, 2 y 4
* Máster 2016: todos
* Máster 2015: sólo 3 y 5

## Ejercicio 1.

1. Mediante Pandas y haciendo uso de la librería ` pandas.io.data.DataReader` obten los datos de prueba para los simbolos `GOOG`, `AAPL` y `MSFT`.
2. Guarda los objetos de Pandas en formato CSV separado por comas.
3. Escribe un pequeño programa en Spark que lea los datos, separe por comas y elimine la primera de las filas en el caso de que el CSV contenga los nombres de las columnas.
4. Mediante Spark, filtra los datos de los años 2015 y 2016.
5. Filtra los datos correspondientes a enero de 2016 mediante Pandas.

Recuerda que puedes descargar un trozo del archivo como referencia mediante `curl` y `head`, desde Yahoo:

```
$ curl --silent wget ichart.yahoo.com/table.csv?s=GOOG | head
Date,Open,High,Low,Close,Volume,Adj Close
2016-07-08,699.50,705.710022,696.434998,705.630005,1571400,705.630005
2016-07-07,698.080017,698.200012,688.215027,695.359985,1303100,695.359985
2016-07-06,689.97998,701.679993,689.090027,697.77002,1408600,697.77002
```

## Ejercicio 2

Haciendo uso únicamente de Spark / Spark SQL:

1. Carga los datos para `MSFT` desde archivo descargado en el ejercicio 1.
2. Obtén el retorno entre los días 15 y 16 de junio del 2016. Recuerda que el retorno entre $t-1$ y $t$ lo definimos como $R_t = \frac{P_t - P_{t-1}}{P_{t-1}}$

In [1]:
%matplotlib inline
import pandas as pd
import pandas.io.data as web
from datetime import datetime
msft = web.DataReader("MSFT", "google", datetime(2015, 1, 1), datetime(2016, 12, 31))
msft.Close["2016-06-15":"2016-06-16"].pct_change()[-1:][0]

0.014087341517408092

## Ejercicio 3

**Importante: sólo alumnos de máster.**

Haciendo uso únicamente de Spark / Spark SQL, extrae una media del precio de cierre por cada cuatrimestre para 2015 y 2016.

In [2]:
msft.Close.resample("Q").mean()

Date
2015-03-31    43.535246
2015-06-30    45.628095
2015-09-30    44.904375
2015-12-31    52.622187
2016-03-31    52.421803
2016-06-30    51.964687
2016-09-30    54.102000
Freq: Q-DEC, Name: Close, dtype: float64

## Ejercicio 4

Haciendo uso únicamente de Spark / Spark SQL, computa el valor de un portfolio a final de mes. Recuerda que el retorno del porfolio para un periodo se define como:

$R_{p,t} = \sum\limits_{i=1}^n x_i R_{i,t}$

donde el porcentaje de participación en el portfolio es $x_n : x_1 + ... + x_n = 1$ y $R_{i,t}$ es cada retorno mensual de cada activo del portfolio.

Conociendo esto es posible extraer el valor del portfolio al final del mes $t$, donde $t$ es

$V_t = V_{t-1}(1 + R_{p,t})$


El portfolio de inicio tiene 10 acciones de `MSFT` y 10 acciones de `AAPL`, ambas compradas el 30 de marzo.

In [3]:
msft = web.DataReader("MSFT", "google", datetime(2015, 12, 25), datetime(2016, 12, 31))
aapl = web.DataReader("AAPL", "google", datetime(2015, 12, 25), datetime(2016, 12, 31))
[msft.Close['2016-03-30'], aapl.Close['2016-03-30']]

[55.049999999999997, 109.56]

Los precios al final de més, donde computaremos el valor del portfolio, son:

In [4]:
[msft.Close['2016-04-29'], aapl.Close['2016-04-29']]

[49.869999999999997, 93.739999999999995]

Por lo que podemos calcular `initial_portfolio_value` de forma simple:

In [5]:
msft_shares = 10
aapl_shares = 10
initial_portfolio_value = (msft_shares * msft.Close['2016-03-30']) + (aapl_shares * aapl.Close['2016-03-30'])
initial_portfolio_value

1646.0999999999999

También necesitamos computar el peso relativo de cada inversión en el total. Observa que sumando ambos pesos nos da un 100% (puede resultarte interesante realizar esta comprobación en Spark):

In [6]:
x_msft = (msft_shares * msft.Close['2016-03-30']) / initial_portfolio_value
x_aapl = (aapl_shares * aapl.Close['2016-03-30']) / initial_portfolio_value
[x_msft, x_aapl, x_msft+x_aapl]

[0.3344268270457445, 0.66557317295425544, 1.0]

Desde aqui solo nos queda calcular el retorno de cada uno de los activos y el del propio portfolio:

In [7]:
ret_msft = msft.Close['2016-04-29'] / msft.Close['2016-03-30'] - 1
ret_aapl = aapl.Close['2016-04-29'] / aapl.Close['2016-03-30'] - 1
[ret_msft, ret_aapl]

[-0.094096276112624833, -0.14439576487769268]

In [8]:
portfolio_returns = (x_msft*ret_msft) + (x_aapl*ret_aapl)
portfolio_returns

-0.12757426644796796

Por lo tanto, el valor del portfolio al final del mes $t$ se puede obtener mediante:

In [9]:
vt = initial_portfolio_value * (1 + portfolio_returns)
vt

1436.0999999999999

## Ejercicio 5

**Importante: sólo alumnos de máster.**

Haciendo uso de Spark, computa la pérdida máxima sobre una cartera con un valor X y una probabilidad del 5%, para una fecha determinada.

El _valor en riesgo_ de una cartera es una medida que nos permite cuantificar las pérdidas en una inversión haciendo uso de herramientas estadísticas básicas. Para simplificar el ejercicio asumiremos que nuestra cartera esta compuesta de 1000 acciones de `AAPL` compradas a fecha de cierre del día 2 de febrero de 2015, por un valor de 106271.212€.

En este ejercicio calcularemos cúanto perderíamos el día siguiente con una probabilidad del 95%. Estadísticamente VaR se puede definir como:

${VaR} = P - P \cdot (\alpha(1-c) + 1)$, siendo $c$ el intervalo de confianza

Lo primero que haremos será obtener los precios históricos de cierre para `AAPL` desde Yahoo Finance.

In [10]:
from pandas.io.data import DataReader
aapl = DataReader('aapl', 'yahoo')
prices = aapl["Adj Close"]
prices["2015-01-02"]

106.27121200000001

A partir de aqui podemos calcular el valor del portfolio en esa fecha. Como el porfolio está compuesto de un solo activo, el cálculo es muy sencillo:

In [11]:
shares = 1000
portfolio_value = shares * prices.ix['2015-1-02']
portfolio_value

106271.212

Lo siguiente que realizaremos será computar los retornos diarios para toda la serie de valores historicos. Recuerda que estamos utilizando la columna `Adj Close`.

In [12]:
import numpy as np

def daily_returns(closes):
    return np.log(closes / closes.shift(1))

returns = daily_returns(prices)
returns[-3:]

Date
2016-07-27    0.062940
2016-07-28    0.013411
2016-07-29   -0.001247
Name: Adj Close, dtype: float64

Desde aqui únicamente nos queda extraer el z-score a aplicar y sustituir valores en la fórmula de VaR descrita arriba:

In [13]:
from scipy.stats import norm
z_score = norm.ppf(0.95) # loc:0 (µ), scale: 1 (σ)
alpha = z_score * returns.std()
value_at_risk = portfolio_value * alpha
value_at_risk

2939.4127308934367

Para simplificar el ejercicio puedes utilizar una constante para `z-score` en lugar de calcularlo dentro del proceso. Igualmente es importante que no hagas uso de numpy o scipy para realizar los cálculos en Spark.